In [2]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict

# ------------------------------------------------
# Load NVIDIA API Key
# ------------------------------------------------
load_dotenv()
nvidia_api = os.getenv("NVIDIA_API_KEY")
assert nvidia_api, "NVIDIA_API_KEY missing"

# ------------------------------------------------
# Load CSV into SQLite
# ------------------------------------------------
data = pd.read_csv("Superstore.csv", encoding="latin1")
conn = sqlite3.connect("sales_data.db")
data.to_sql("sales", conn, if_exists="replace", index=False)
print("✅ Loaded Superstore.csv → SQLite (table: sales)")

# ------------------------------------------------
# Initialize NVIDIA LLM
# ------------------------------------------------
llm = ChatNVIDIA(
    model="meta/llama-3.1-70b-instruct",
    temperature=0.1,
    api_key=nvidia_api
)

# ------------------------------------------------
# Define State Structure
# ------------------------------------------------
class State(TypedDict):
    messages: List[Dict]
    intent: str
    sql_query: str
    sql_results: str
    insight: str

# ------------------------------------------------
# Node 1 — INTENT ANALYSIS
# ------------------------------------------------
def intent_node(state: State):
    user_msg = state["messages"][-1]["content"]

    prompt = f"""
Classify the user's intent as:

SQL → If user asks for data, totals, monthly trends, grouped values.
INSIGHT → If user asks for explanation, seasonality, why something happened.

User query: "{user_msg}"

Respond ONLY: SQL or INSIGHT
"""

    intent = llm.invoke([{"role": "user", "content": prompt}]).content.strip().upper()

    return {**state, "intent": intent}

# ------------------------------------------------
# Node 2 — SQL GENERATOR
# ------------------------------------------------
def sql_generator_node(state: State):
    if state["intent"] != "SQL":
        return state

    user_msg = state["messages"][-1]["content"]

    columns = ", ".join([f'"{c}"' for c in data.columns])

    prompt = f"""
You are an elite SQL generator. Output ONLY a valid SQLite query. No explanations.

TABLE: sales  
COLUMNS: {columns}

DATE FORMAT RULE:
"Order Date" is dd/mm/yyyy → convert using:
SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2)

User request:
{user_msg}

Output only SQL:
"""

    sql = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()

    return {**state, "sql_query": sql}

# ------------------------------------------------
# Node 3 — SQL EXECUTION
# ------------------------------------------------
def sql_executor_node(state: State):
    sql = state.get("sql_query")
    if not sql:
        return state

    cursor = conn.cursor()

    try:
        cursor.execute(sql)
        rows = cursor.fetchall()
        cols = [desc[0] for desc in cursor.description]

        if rows:
            formatted = "\n".join(
                ", ".join(f"{c}: {v}" for c, v in zip(cols, row))
                for row in rows
            )
        else:
            formatted = "No results."

    except Exception as e:
        formatted = f"SQL ERROR: {e}"

    return {**state, "sql_results": formatted}

# ------------------------------------------------
# Node 4 — INSIGHT ANALYSIS
# ------------------------------------------------
def insight_node(state: State):
    user_msg = state["messages"][-1]["content"]

    sql_results = state.get("sql_results", "")

    prompt = f"""
You are a senior business analyst.

Explain insights from the SQL results.
User question: {user_msg}

SQL results:
{sql_results}

Provide:
- Monthly patterns
- Peaks & lows
- Seasonality
- Business explanation
"""

    insight = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    return {**state, "insight": insight}

# ------------------------------------------------
# Node 5 — FINAL OUTPUT
# ------------------------------------------------
def output_node(state: State):
    result = f"""
 **Full Analysis**

 **SQL Query**
{state.get("sql_query","")}

 **SQL Results**
{state.get("sql_results","")}

 **Analysis & Insights**
{state.get("insight","")}
"""

    return {
        **state,
        "messages": state["messages"] + [{"role": "assistant", "content": result}]
    }

# ------------------------------------------------
# Build LangGraph
# ------------------------------------------------
graph = StateGraph(State)

graph.add_node("intent", intent_node)
graph.add_node("sql_gen", sql_generator_node)
graph.add_node("sql_exec", sql_executor_node)
graph.add_node("insight", insight_node)
graph.add_node("output", output_node)

graph.set_entry_point("intent")

graph.add_edge("intent", "sql_gen")
graph.add_edge("sql_gen", "sql_exec")
graph.add_edge("sql_exec", "insight")
graph.add_edge("insight", "output")
graph.add_edge("output", END)

app = graph.compile()

# ------------------------------------------------
# Run Chat Loop
# ------------------------------------------------
if __name__ == "__main__":
    print("💬 Ask anything about your dataset. Example:")
    print("👉 What is the seasonal trend in office supplies in 2015?")
    print("Type 'exit' to quit.\n")

    while True:
        q = input("You: ").strip()
        if q.lower() == "exit":
            break

        conversation = {"messages": [{"role": "user", "content": q}]}
        output = app.invoke(conversation)
        print("Assistant:", output["messages"][-1]["content"])


✅ Loaded Superstore.csv → SQLite (table: sales)
💬 Ask anything about your dataset. Example:
👉 What is the seasonal trend in office supplies in 2015?
Type 'exit' to quit.

Assistant: 
 **Full Analysis**

 **SQL Query**


 **SQL Results**


 **Analysis & Insights**
Based on the SQL results, I'll provide insights into the seasonal trend in the consumer segment for 2015.

**Monthly Patterns:**

The data shows a consistent pattern of sales throughout the year, with some fluctuations. Here's a breakdown of the monthly sales:

* January: 85,000 units sold
* February: 78,000 units sold
* March: 82,000 units sold
* April: 90,000 units sold
* May: 95,000 units sold
* June: 100,000 units sold
* July: 105,000 units sold
* August: 110,000 units sold
* September: 100,000 units sold
* October: 95,000 units sold
* November: 90,000 units sold
* December: 85,000 units sold

**Peaks & Lows:**

The data indicates that:

* The peak sales month is August with 110,000 units sold.
* The lowest sales month is 

In [14]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict

# ------------------------------------------------
# Load NVIDIA API Key
# ------------------------------------------------
load_dotenv()
nvidia_api = os.getenv("NVIDIA_API_KEY")
assert nvidia_api, "NVIDIA_API_KEY missing"

# ------------------------------------------------
# Load CSV into SQLite
# ------------------------------------------------
data = pd.read_csv("Superstore.csv", encoding="latin1")
conn = sqlite3.connect("sales_data.db")
data.to_sql("sales", conn, if_exists="replace", index=False)
print("✅ Loaded Superstore.csv → SQLite (table: sales)")

# ------------------------------------------------
# Initialize NVIDIA LLM
# ------------------------------------------------
llm = ChatNVIDIA(
    model="meta/llama-3.1-70b-instruct",
    temperature=0.1,
    api_key=nvidia_api
)

# ------------------------------------------------
# Define State Structure
# ------------------------------------------------
class State(TypedDict):
    messages: List[Dict]
    intent: str
    sql_query: str
    sql_results: str
    insight: str

# ------------------------------------------------
# Node 1 — INTENT ANALYSIS
# ------------------------------------------------
def intent_node(state: State):
    user_msg = state["messages"][-1]["content"]

    prompt = f"""
Classify the user's intent as:

SQL → If user asks for data, totals, monthly trends, grouped values.
INSIGHT → If user asks for explanation, seasonality, why something happened.

User query: "{user_msg}"

Respond ONLY: SQL or INSIGHT
"""

    intent = llm.invoke([{"role": "user", "content": prompt}]).content.strip().upper()

    return {**state, "intent": intent}

# ------------------------------------------------
# Node 2 — SQL GENERATOR
# ------------------------------------------------
def sql_generator_node(state: State):
    if state["intent"] != "SQL":
        return state

    user_msg = state["messages"][-1]["content"]

    columns = ", ".join([f'"{c}"' for c in data.columns])

    prompt = f"""
You are an elite SQL generator. Output ONLY a valid SQLite query. No explanations.

TABLE: sales  
COLUMNS: {columns}

DATE FORMAT RULE:
"Order Date" is dd/mm/yyyy → convert using:
SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2)

User request:
{user_msg}

Output only SQL:
"""

    sql = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()

    return {**state, "sql_query": sql}

# ------------------------------------------------
# Node 3 — SQL EXECUTION
# ------------------------------------------------
def sql_executor_node(state: State):
    sql = state.get("sql_query")
    if not sql:
        return state

    cursor = conn.cursor()

    try:
        cursor.execute(sql)
        rows = cursor.fetchall()
        cols = [desc[0] for desc in cursor.description]

        if rows:
            formatted = "\n".join(
                ", ".join(f"{c}: {v}" for c, v in zip(cols, row))
                for row in rows
            )
        else:
            formatted = "No results."

    except Exception as e:
        formatted = f"SQL ERROR: {e}"

    return {**state, "sql_results": formatted}

# ------------------------------------------------
# Node 4 — INSIGHT ANALYSIS
# ------------------------------------------------
def insight_node(state: State):
    user_msg = state["messages"][-1]["content"]

    sql_results = state.get("sql_results", "")

    prompt = f"""
You are a senior business analyst.

Explain insights from the SQL results.
User question: {user_msg}

SQL results:
{sql_results}

Provide:
- Monthly patterns
- Peaks & lows
- Seasonality
- Business explanation
"""

    insight = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    return {**state, "insight": insight}

# ------------------------------------------------
# Node 5 — FINAL OUTPUT
# ------------------------------------------------
def output_node(state: State):
    result = f"""
 **Full Analysis**

 **SQL Query**
{state.get("sql_query","")}

 **SQL Results**
{state.get("sql_results","")}

 **Analysis & Insights**
{state.get("insight","")}
"""

    return {
        **state,
        "messages": state["messages"] + [{"role": "assistant", "content": result}]
    }

# ------------------------------------------------
# Build LangGraph
# ------------------------------------------------
graph = StateGraph(State)

graph.add_node("intent", intent_node)
graph.add_node("sql_gen", sql_generator_node)
graph.add_node("sql_exec", sql_executor_node)
graph.add_node("insight", insight_node)
graph.add_node("output", output_node)

graph.set_entry_point("intent")

graph.add_edge("intent", "sql_gen")
graph.add_edge("sql_gen", "sql_exec")
graph.add_edge("sql_exec", "insight")
graph.add_edge("insight", "output")
graph.add_edge("output", END)

app = graph.compile()

# ------------------------------------------------
# Run Chat Loop
# ------------------------------------------------
if __name__ == "__main__":
    print("💬 Ask anything about your dataset. Example:")
    print("👉 What is the seasonal trend in office supplies in 2015?")
    print("Type 'exit' to quit.\n")

    while True:
        q = input("You: ").strip()
        if q.lower() == "exit":
            break

        conversation = {"messages": [{"role": "user", "content": q}]}
        output = app.invoke(conversation)
        print("Assistant:", output["messages"][-1]["content"])



✅ Loaded Superstore.csv → SQLite (table: sales)
💬 Ask anything about your dataset. Example:
👉 What is the seasonal trend in office supplies in 2015?
Type 'exit' to quit.

Assistant: 
 **Full Analysis**

 **SQL Query**
SELECT "Product Name", SUM("Sales") FROM sales WHERE SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2) LIKE '2015%' AND "Category" = 'Office Supplies' GROUP BY "Product Name"

 **SQL Results**
Product Name: #10- 4 1/8" x 9 1/2" Recycled Envelopes, SUM("Sales"): 73.416
Product Name: #10-4 1/8" x 9 1/2" Premium Diagonal Seam Envelopes, SUM("Sales"): 113.328
Product Name: #6 3/4 Gummed Flap White Envelopes, SUM("Sales"): 7.92
Product Name: 1/4 Fold Party Design Invitations & White Envelopes, 24 8-1/2" X 11" Cards, 25 Env./Pack, SUM("Sales"): 44.1
Product Name: 2300 Heavy-Duty Transfer File Systems by Perma, SUM("Sales"): 74.94
Product Name: 3-ring staple pack, SUM("Sales"): 3.76
Product Name: 3M Office Air Cleaner, SUM("Sales"): 103.92
Pr

In [ ]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict
import plotly.express as px
import uuid
import webbrowser
from dateutil import parser
import glob

# ------------------------
# Load NVIDIA API Key
# ------------------------
load_dotenv()
nvidia_api = os.getenv("NVIDIA_API_KEY")
assert nvidia_api, "NVIDIA_API_KEY missing"

# ------------------------
# Load CSV into SQLite
# ------------------------
data = pd.read_csv("Superstore.csv", encoding="latin1")

# Normalize dates
def normalize_date(date_str):
    try:
        return parser.parse(str(date_str)).strftime("%Y-%m-%d")
    except:
        return None

if "Order Date" in data.columns:
    data["Order Date"] = data["Order Date"].apply(normalize_date)

conn = sqlite3.connect("sales_data.db")
data.to_sql("sales", conn, if_exists="replace", index=False)
print("✅ Loaded Superstore.csv → SQLite (table: sales)")

# ------------------------
# Initialize LLaMA (NVIDIA)
# ------------------------
llm = ChatNVIDIA(
    model="meta/llama-3.1-70b-instruct",
    temperature=0.1,
    api_key=nvidia_api
)

# ------------------------
# Define LangGraph State
# ------------------------
class State(TypedDict):
    messages: List[Dict]
    intent: str
    sql_query: str
    sql_results: str
    insight: str
    df_sql: pd.DataFrame
    chart_path: str

# ------------------------
# Node functions
# ------------------------
def intent_node(state: State):
    user_msg = state["messages"][-1]["content"]
    prompt = f"""
Classify the user's intent as:

SQL → If user asks for data, totals, monthly trends, grouped values.
INSIGHT → If user asks for explanation, seasonality, why something happened.

User query: "{user_msg}"

Respond ONLY: SQL or INSIGHT
"""
    intent = llm.invoke([{"role": "user", "content": prompt}]).content.strip().upper()
    return {**state, "intent": intent}

def sql_generator_node(state: State):
    if state["intent"] != "SQL":
        return state
    user_msg = state["messages"][-1]["content"]
    columns = ", ".join([f'"{c}"' for c in data.columns])
    prompt = f"""
You are an expert SQL generator. Output ONLY valid SQLite query. No explanations.

TABLE: sales  
COLUMNS: {columns}

DATE FORMAT RULE:
"Order Date" is normalized to yyyy-mm-dd

User request:
{user_msg}

Output only SQL:
"""
    sql = llm.invoke([{"role": "user", "content": prompt}]).content.strip()
    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()
    return {**state, "sql_query": sql}

def sql_executor_node(state: State):
    sql = state.get("sql_query")
    if not sql:
        return state
    df_sql = pd.DataFrame()
    formatted = ""
    try:
        df_sql = pd.read_sql(sql, conn)
        formatted = df_sql.head(5).to_string(index=False)
    except Exception as e:
        formatted = f"SQL ERROR: {e}"
    return {**state, "sql_results": formatted, "df_sql": df_sql}

def chart_generator_node(state: State):
    df_sql = state.get("df_sql")
    chart_path = ""
    if df_sql is not None and not df_sql.empty:
        numeric_cols = df_sql.select_dtypes(include='number').columns
        if len(numeric_cols) > 0:
            col = numeric_cols[0]
            cat_cols = df_sql.select_dtypes(include='object').columns
            if len(cat_cols) > 0:
                x_col = cat_cols[0]
                fig = px.bar(df_sql, x=x_col, y=col, title=f"{col} by {x_col}", text=col)
                fig.update_layout(xaxis_tickangle=-45)
            else:
                fig = px.line(df_sql, y=col, title=f"{col} over rows")
            chart_path = f"chart_{uuid.uuid4().hex}.html"
            fig.write_html(chart_path)
            webbrowser.open(f"file://{os.path.abspath(chart_path)}")
    return {**state, "chart_path": chart_path}

def insight_node(state: State):
    user_msg = state["messages"][-1]["content"]
    sql_results = state.get("sql_results", "")
    if state["intent"] == "INSIGHT":
        prompt = f"""
You are a senior business analyst.

Explain insights from the SQL results.
User question: {user_msg}

SQL results:
{sql_results}

Provide:
- Monthly patterns (if applicable)
- Peaks & lows
- Seasonality
- Business explanation
"""
        insight = llm.invoke([{"role": "user", "content": prompt}]).content.strip()
    else:
        insight = ""
    return {**state, "insight": insight}

def output_node(state: State):
    chart_info = (
        f" Interactive chart available: [Open Chart]({state['chart_path']})"
        if state.get("chart_path") else "No chart generated."
    )
    result = f"""
 **Full Analysis**

 **SQL Query**
{state.get("sql_query","")}

 **SQL Results**
{state.get("sql_results","")}

 **Analysis & Insights**
{state.get("insight","")}

{chart_info}
"""
    return {**state, "messages": state["messages"] + [{"role": "assistant", "content": result}]}

# ------------------------
# Function to initialize fresh LangGraph app
# ------------------------
def initialize_app():
    graph = StateGraph(State)
    graph.add_node("intent", intent_node)
    graph.add_node("sql_gen", sql_generator_node)
    graph.add_node("sql_exec", sql_executor_node)
    graph.add_node("chart", chart_generator_node)
    graph.add_node("insight", insight_node)
    graph.add_node("output", output_node)
    graph.set_entry_point("intent")
    graph.add_edge("intent", "sql_gen")
    graph.add_edge("sql_gen", "sql_exec")
    graph.add_edge("sql_exec", "chart")
    graph.add_edge("chart", "insight")
    graph.add_edge("insight", "output")
    graph.add_edge("output", END)
    return graph.compile()

# ------------------------
# Chat Loop
# ------------------------
if __name__ == "__main__":
    print("💬 Ask anything about your dataset. Example:")
    print("👉 What is the seasonal trend in office supplies in 2015?")
    print("Type 'exit' to quit.\n")

    while True:
        q = input("You: ").strip()
        if q.lower() == "exit":
            # Clean up all generated chart HTML files
            for f in glob.glob("chart_*.html"):
                try:
                    os.remove(f)
                except:
                    pass
            break

        # Initialize fresh LangGraph app for each new input
        app = initialize_app()
        conversation = {"messages": [{"role": "user", "content": q}]}
        output = app.invoke(conversation)
        print("Assistant:", output["messages"][-1]["content"])


✅ Loaded Superstore.csv → SQLite (table: sales)
💬 Ask anything about your dataset. Example:
👉 What is the seasonal trend in office supplies in 2015?
Type 'exit' to quit.

Assistant: 
 **Full Analysis**

 **SQL Query**
SELECT "Product Name", SUM("Sales") FROM sales WHERE "Category" = 'Office Supplies' AND STRFTIME('%Y', "Order Date") = '2015' GROUP BY "Product Name"

 **SQL Results**
                                       Product Name  SUM("Sales")
           #10 Gummed Flap White Envelopes, 100/Box         6.608
         #10 White Business Envelopes,4 1/8 x 9 1/2        12.536
            #10- 4 1/8" x 9 1/2" Recycled Envelopes       134.596
#10-4 1/8" x 9 1/2" Premium Diagonal Seam Envelopes       113.328
                 #6 3/4 Gummed Flap White Envelopes        23.760

 **Analysis & Insights**


📊 Interactive chart available: [Open Chart](chart_215327ec690f439ca1a64e6e311908b0.html)

Assistant: 
 **Full Analysis**

 **SQL Query**
SELECT "Product Name", SUM("Sales") FROM sales WHERE 

In [18]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict

# ------------------------------------------------
# Load NVIDIA API Key
# ------------------------------------------------
load_dotenv()
nvidia_api = os.getenv("NVIDIA_API_KEY")
assert nvidia_api, "NVIDIA_API_KEY missing"

# ------------------------------------------------
# Load CSV into SQLite
# ------------------------------------------------
data = pd.read_csv("Superstore.csv", encoding="latin1")
conn = sqlite3.connect("sales_data.db")
data.to_sql("sales", conn, if_exists="replace", index=False)
print("✅ Loaded Superstore.csv → SQLite (table: sales)")

# ------------------------------------------------
# Initialize NVIDIA LLM
# ------------------------------------------------
llm = ChatNVIDIA(
    model="meta/llama-3.1-70b-instruct",
    temperature=0.1,
    api_key=nvidia_api
)

# ------------------------------------------------
# Define State Structure
# ------------------------------------------------
class State(TypedDict):
    messages: List[Dict]
    intent: str
    sql_query: str
    sql_results: str
    insight: str

# ------------------------------------------------
# Node 1 — INTENT ANALYSIS
# ------------------------------------------------
def intent_node(state: State):
    user_msg = state["messages"][-1]["content"]

    prompt = f"""
Classify the user's intent as:

SQL → If user asks for data, totals, monthly trends, grouped values.
INSIGHT → If user asks for explanation, seasonality, why something happened.

User query: "{user_msg}"

Respond ONLY: SQL or INSIGHT
"""

    intent = llm.invoke([{"role": "user", "content": prompt}]).content.strip().upper()

    return {**state, "intent": intent}

# ------------------------------------------------
# Node 2 — SQL GENERATOR
# ------------------------------------------------
def sql_generator_node(state: State):
    if state["intent"] != "SQL":
        return state

    user_msg = state["messages"][-1]["content"]

    columns = ", ".join([f'"{c}"' for c in data.columns])

    prompt = f"""
You are an elite SQL generator. Output ONLY a valid SQLite query. No explanations.

TABLE: sales  
COLUMNS: {columns}

DATE FORMAT RULE:
"Order Date" is dd/mm/yyyy → convert using:
SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2)

User request:
{user_msg}

Output only SQL:
"""

    sql = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()

    return {**state, "sql_query": sql}

# ------------------------------------------------
# Node 3 — SQL EXECUTION
# ------------------------------------------------
def sql_executor_node(state: State):
    sql = state.get("sql_query")
    if not sql:
        return state

    cursor = conn.cursor()

    try:
        cursor.execute(sql)
        rows = cursor.fetchall()
        cols = [desc[0] for desc in cursor.description]

        if rows:
            formatted = "\n".join(
                ", ".join(f"{c}: {v}" for c, v in zip(cols, row))
                for row in rows
            )
        else:
            formatted = "No results."

    except Exception as e:
        formatted = f"SQL ERROR: {e}"

    return {**state, "sql_results": formatted}

# ------------------------------------------------
# Node 4 — INSIGHT ANALYSIS
# ------------------------------------------------
def insight_node(state: State):
    user_msg = state["messages"][-1]["content"]

    sql_results = state.get("sql_results", "")

    prompt = f"""
You are a senior business analyst.

Explain insights from the SQL results.
User question: {user_msg}

SQL results:
{sql_results}

Provide:
- Monthly patterns
- Peaks & lows
- Seasonality
- Business explanation
"""

    insight = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    return {**state, "insight": insight}

# ------------------------------------------------
# Node 5 — FINAL OUTPUT
# ------------------------------------------------
def output_node(state: State):
    result = f"""
 **Full Analysis**

 **SQL Query**
{state.get("sql_query","")}

 **SQL Results**
{state.get("sql_results","")}

 **Analysis & Insights**
{state.get("insight","")}
"""

    return {
        **state,
        "messages": state["messages"] + [{"role": "assistant", "content": result}]
    }

# ------------------------------------------------
# Build LangGraph
# ------------------------------------------------
graph = StateGraph(State)

graph.add_node("intent", intent_node)
graph.add_node("sql_gen", sql_generator_node)
graph.add_node("sql_exec", sql_executor_node)
graph.add_node("insight", insight_node)
graph.add_node("output", output_node)

graph.set_entry_point("intent")

graph.add_edge("intent", "sql_gen")
graph.add_edge("sql_gen", "sql_exec")
graph.add_edge("sql_exec", "insight")
graph.add_edge("insight", "output")
graph.add_edge("output", END)

app = graph.compile()

# ------------------------------------------------
# Run Chat Loop
# ------------------------------------------------
if __name__ == "__main__":
    print("💬 Ask anything about your dataset. Example:")
    print("👉 What is the seasonal trend in office supplies in 2015?")
    print("Type 'exit' to quit.\n")

    while True:
        q = input("You: ").strip()
        if q.lower() == "exit":
            break

        conversation = {"messages": [{"role": "user", "content": q}]}
        output = app.invoke(conversation)
        print("Assistant:", output["messages"][-1]["content"])



✅ Loaded Superstore.csv → SQLite (table: sales)
💬 Ask anything about your dataset. Example:
👉 What is the seasonal trend in office supplies in 2015?
Type 'exit' to quit.

Assistant: 
 **Full Analysis**

 **SQL Query**
SELECT "Sub-Category", SUM("Sales") FROM sales WHERE "Category" = 'Furniture' AND SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2) LIKE '2015%' GROUP BY "Sub-Category"

 **SQL Results**
Sub-Category: Bookcases, SUM("Sales"): 7704.0102
Sub-Category: Chairs, SUM("Sales"): 16238.663
Sub-Category: Furnishings, SUM("Sales"): 5046.67
Sub-Category: Tables, SUM("Sales"): 11690.422

 **Analysis & Insights**
Based on the SQL results, here are the insights:

**Monthly Patterns:**
Since we don't have monthly data, we can only analyze the yearly sales data. However, we can still identify some patterns:

* The sales data suggests that the sub-categories in the Furniture category have a relatively consistent sales performance throughout the year, wi

In [ ]:
# full_chatbot_final_with_insights.py
import os
import sqlite3
import pandas as pd
from dateutil import parser
import difflib
import plotly.express as px
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict, Any

# -------------------------
# Config
# -------------------------
load_dotenv()
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
assert NVIDIA_API_KEY, "NVIDIA_API_KEY missing in .env"
CSV_FILE = "Superstore.csv"
DB_FILE = "sales_data.db"
MAX_SQL_RETRIES = 3

# -------------------------
# Helpers: normalize dates robustly
# -------------------------
def normalize_date_col(df: pd.DataFrame, col_name: str) -> pd.Series:
    if col_name not in df.columns:
        return None
    def _norm(v):
        try:
            if pd.isna(v):
                return None
            return parser.parse(str(v), dayfirst=True).strftime("%Y-%m-%d")
        except Exception:
            return None
    return df[col_name].apply(_norm)

# -------------------------
# Load dataset, normalize date columns, write to SQLite
# -------------------------
df = pd.read_csv(CSV_FILE, encoding="latin1")
for c in ("Order Date", "Ship Date", "OrderDate", "ShipDate"):
    if c in df.columns:
        df[c] = normalize_date_col(df, c)
for c in list(df.columns):
    if "date" in c.lower() and df[c].dtype == object:
        df[c] = normalize_date_col(df, c)

conn = sqlite3.connect(DB_FILE)
df.to_sql("sales", conn, if_exists="replace", index=False)
print(f"✅ Loaded {CSV_FILE} → {DB_FILE} (table: sales), dates normalized where possible")

# -------------------------
# LLM (NVIDIA-backed LLaMA)
# -------------------------
llm = ChatNVIDIA(model="meta/llama-3.1-70b-instruct", temperature=0.1, api_key=NVIDIA_API_KEY)

# -------------------------
# Utility: get schema + preview for prompt prefeed
# -------------------------
COLUMN_LIST = list(df.columns)
PREVIEW_5 = df.head(5).to_string(index=False)

def prefeed_context() -> str:
    cols = ", ".join([f'"{c}"' for c in COLUMN_LIST])
    return f"COLUMNS: {cols}\nFIRST_5_ROWS:\n{PREVIEW_5}\nNote: 'Order Date' values (if present) are in yyyy-mm-dd."

# -------------------------
# SQL auto-fix utilities
# -------------------------
def find_closest_col(token: str) -> str:
    matches = difflib.get_close_matches(token, COLUMN_LIST, n=1, cutoff=0.6)
    if matches:
        return matches[0]
    low_matches = difflib.get_close_matches(token.lower(), [c.lower() for c in COLUMN_LIST], n=1, cutoff=0.6)
    if low_matches:
        for c in COLUMN_LIST:
            if c.lower() == low_matches[0]:
                return c
    return None

def auto_fix_sql(sql: str, last_error: str) -> str:
    fixed = sql
    if "no such column" in last_error.lower():
        import re
        toks = re.findall(r"no such column: ([\w\"']+)", last_error, flags=re.I)
        if toks:
            for t in toks:
                t_clean = t.strip('"\'')
                closest = find_closest_col(t_clean)
                if closest:
                    fixed = fixed.replace(t_clean, f'"{closest}"')
    for col in COLUMN_LIST:
        if col in fixed and f'"{col}"' not in fixed:
            fixed = fixed.replace(f" {col} ", f' "{col}" ')
            fixed = fixed.replace(f" {col},", f' "{col}",')
            fixed = fixed.replace(f",{col} ", f',"{col}" ')
            fixed = fixed.replace(f"({col} ", f'("{col}" ')
    import re
    year_match = re.search(r"\b(20\d{2}|19\d{2})\b", sql)
    if year_match and "between" not in sql.lower() and '"Order Date"' in fixed:
        y = year_match.group(1)
        if "where" in fixed.lower():
            fixed = re.sub(r"where", f"WHERE", fixed, flags=re.I)
            fixed += f" AND \"Order Date\" >= '{y}-01-01' AND \"Order Date\" <= '{y}-12-31'"
        else:
            fixed += f" WHERE \"Order Date\" >= '{y}-01-01' AND \"Order Date\" <= '{y}-12-31'"
    return fixed

# -------------------------
# SQL attempt helper
# -------------------------
def try_execute_sql(sql: str) -> (bool, str, pd.DataFrame):
    try:
        df_sql = pd.read_sql(sql, conn)
        return True, "", df_sql
    except Exception as e:
        return False, str(e), pd.DataFrame()

# -------------------------
# LangGraph State Type
# -------------------------
class State(TypedDict):
    messages: List[Dict[str, Any]]
    intent: str
    sql_query: str
    sql_results: str
    insight: str
    df: pd.DataFrame
    chart_html: str
    attempts: int

# -------------------------
# Node: Intent classification
# -------------------------
def intent_node(state: State) -> State:
    user_msg = state["messages"][-1]["content"]
    prompt = f"""
Classify the user's intent as exactly one of: SQL, INSIGHT, SQL+CHART.
- SQL => data extraction
- INSIGHT => explanation, reasoning
- SQL+CHART => data + visualization

Context:
{prefeed_context()}

User query: "{user_msg}"

Respond with exactly one token: SQL, INSIGHT, or SQL+CHART
"""
    resp = llm.invoke([{"role":"user","content":prompt}]).content.strip().upper()
    intent = resp.split()[0] if resp else "SQL"
    if intent not in ("SQL","INSIGHT","SQL+CHART"):
        intent = "SQL"

    if any(word in user_msg.lower() for word in ("summary","insight","analyze","review","trend")):
        intent = "SQL+CHART"
    return {**state, "intent": intent}

# -------------------------
# Node: SQL generation
# -------------------------
def sql_generator_node(state: State) -> State:
    if state.get("intent") not in ("SQL","SQL+CHART"):
        return state
    user_msg = state["messages"][-1]["content"]
    base_prompt = f"""
You are an expert SQLite SQL generator. Use table 'sales'.
Prefeed: {prefeed_context()}

User request: {user_msg}

OUTPUT RULES:
- Output only one valid SQL SELECT
- Wrap column names in double quotes
- If year mentioned, use 'Order Date' BETWEEN YYYY-01-01 AND YYYY-12-31
"""
    sql = llm.invoke([{"role":"user","content":base_prompt}]).content.strip()
    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()
    return {**state, "sql_query": sql, "attempts": 0}

# -------------------------
# Node: SQL executor with retries
# -------------------------
def sql_executor_node(state: State) -> State:
    sql = state.get("sql_query","")
    if not sql:
        return state
    attempts = 0
    df_sql = pd.DataFrame()
    last_err = ""
    current_sql = sql
    while attempts < MAX_SQL_RETRIES:
        attempts += 1
        ok, err, df_try = try_execute_sql(current_sql)
        if ok:
            df_sql = df_try
            last_err = ""
            break
        last_err = err
        current_sql = auto_fix_sql(current_sql, last_err)
        if current_sql.strip() == sql.strip() or attempts == 1:
            fix_prompt = f"""
SQL: {current_sql}
ERROR: {last_err}

Provide a corrected SQLite SELECT statement only.
"""
            fixed = llm.invoke([{"role":"user","content":fix_prompt}]).content.strip()
            if "select" in fixed.lower():
                current_sql = fixed[fixed.lower().index("select"):].strip()
    if df_sql.empty and last_err:
        formatted = f"SQL could not be executed after {attempts} attempts. Last error: {last_err}"
        return {**state, "sql_results": formatted, "df": pd.DataFrame(), "attempts": attempts}
    formatted = df_sql.head(20).to_string(index=False)
    return {**state, "sql_results": formatted, "df": df_sql, "attempts": attempts}

# -------------------------
# Node: Chart generation
# -------------------------
def chart_node(state: State) -> State:
    df_sql = state.get("df")
    if df_sql is None or df_sql.empty:
        return {**state, "chart_html": ""}
    date_col = next((c for c in df_sql.columns if "date" in c.lower()), None)
    numeric_cols = df_sql.select_dtypes(include="number").columns.tolist()
    cat_cols = df_sql.select_dtypes(include="object").columns.tolist()
    fig = None
    try:
        if date_col and numeric_cols:
            df_sql[date_col] = pd.to_datetime(df_sql[date_col], errors="coerce")
            fig = px.line(df_sql.sort_values(by=date_col), x=date_col, y=numeric_cols[0],
                          title=f"{numeric_cols[0]} over {date_col}")
        elif cat_cols and numeric_cols:
            x = cat_cols[0]
            y = numeric_cols[0]
            if df_sql.shape[0] > 10:
                df_plot = df_sql.groupby(x)[y].sum().reset_index().sort_values(y, ascending=True)
                fig = px.bar(df_plot, x=y, y=x, orientation="h", title=f"{y} by {x} (top grouped)")
            else:
                fig = px.bar(df_sql, x=x, y=y, title=f"{y} by {x}")
        elif len(numeric_cols) >= 2:
            fig = px.scatter(df_sql, x=numeric_cols[0], y=numeric_cols[1], title=f"{numeric_cols[1]} vs {numeric_cols[0]}")
        elif len(numeric_cols) == 1:
            fig = px.histogram(df_sql, x=numeric_cols[0], nbins=20, title=f"Distribution of {numeric_cols[0]}")
        else:
            if df_sql.shape[1] >= 2:
                x = df_sql.columns[0]
                y = df_sql.columns[1]
                try:
                    df_sql[y] = pd.to_numeric(df_sql[y], errors="coerce")
                    fig = px.bar(df_sql, x=x, y=y, title=f"{y} by {x}")
                except:
                    fig = None
    except:
        fig = None
    chart_html = fig.to_html(full_html=False, include_plotlyjs='cdn') if fig else ""
    return {**state, "chart_html": chart_html}

# -------------------------
# Node: Insight generation
# -------------------------
def insight_node(state: State) -> State:
    df_sql = state.get("df")
    sql_results = state.get("sql_results","")
    if sql_results.startswith("SQL could not be executed") or df_sql is None or df_sql.empty:
        return {**state, "insight": " No data returned. No insights can be generated."}
    numeric_summary = df_sql.describe(include="all").head(10).to_string()
    prompt = f"""
You are a business analyst. Use ONLY this data.

User query: {state['messages'][-1]['content']}

Numeric summary and first 10 rows:
{numeric_summary}

First 10 rows:
{df_sql.head(10).to_string(index=False)}

Instructions:
1. Short summary (1-3 sentences) of numeric findings
2. Top/bottom items explicitly
3. Monthly/seasonal patterns if date column exists
4. Do NOT invent numbers
"""
    resp = llm.invoke([{"role":"user","content":prompt}]).content.strip()
    return {**state, "insight": resp}

# -------------------------
# Node: Output
# -------------------------
def output_node(state: State) -> State:
    chart_html = state.get("chart_html","")
    result_lines = [
        " **Full Analysis**",
        "\n **SQL Query**",
        state.get("sql_query","(no SQL)"),
        "\n **SQL Results (preview)**",
        state.get("sql_results","(no results)"),
        "\n **Analysis & Insights**",
        state.get("insight","(no insights)")
    ]
    if chart_html:
        result_lines.append("\n **Interactive Chart (inline)**\n")
        result_lines.append(chart_html)
        result_lines.append("\n(Note: use the Plotly toolbar to download PNG.)")
    final = "\n".join(result_lines)
    return {**state, "messages": state["messages"] + [{"role":"assistant","content": final}]}

# -------------------------
# Initialize LangGraph app
# -------------------------
def initialize_app():
    graph = StateGraph(State)
    graph.add_node("intent", intent_node)
    graph.add_node("sql_gen", sql_generator_node)
    graph.add_node("sql_exec", sql_executor_node)
    graph.add_node("chart", chart_node)
    graph.add_node("insight", insight_node)
    graph.add_node("output", output_node)
    graph.set_entry_point("intent")
    graph.add_edge("intent", "sql_gen")
    graph.add_edge("sql_gen", "sql_exec")
    graph.add_edge("sql_exec", "chart")
    graph.add_edge("chart", "insight")
    graph.add_edge("insight", "output")
    graph.add_edge("output", END)
    return graph.compile()

# -------------------------
# Main chat loop
# -------------------------
def main():
    print("💬 Chatbot ready. Type 'exit' to quit.")
    while True:
        q = input("\nYou: ").strip()
        if q.lower() == "exit":
            print("Session closed. LangGraph state cleared.")
            break
        app = initialize_app()
        conv = {"messages":[{"role":"user","content":q}]}
        out = app.invoke(conv)
        assistant_content = out["messages"][-1]["content"]
        print("\nAssistant:\n")
        print(assistant_content)
        print("\n" + "-"*80)

if __name__ == "__main__":
    main()


✅ Loaded Superstore.csv → sales_data.db (table: sales), dates normalized where possible
💬 Chatbot ready. Type 'exit' to quit.

Assistant:

 **Full Analysis**

 **SQL Query**
SELECT "Product Name", SUM("Sales") FROM sales WHERE "Category" = 'Office Supplies' AND "Order Date" BETWEEN '2015-01-01' AND '2015-12-31' GROUP BY "Product Name"

 **SQL Results (preview)**
                                                                            Product Name  SUM("Sales")
                                                #10 Gummed Flap White Envelopes, 100/Box         6.608
                                              #10 White Business Envelopes,4 1/8 x 9 1/2        12.536
                                                 #10- 4 1/8" x 9 1/2" Recycled Envelopes       134.596
                                     #10-4 1/8" x 9 1/2" Premium Diagonal Seam Envelopes       113.328
                                                      #6 3/4 Gummed Flap White Envelopes        23.760
1/4 Fold Party De

In [ ]:
import os
import sqlite3
import pandas as pd
import streamlit as st
import plotly.express as px
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict
from typing import List, Dict, Any

# ------------------------------------------------
# Load NVIDIA API Key
# ------------------------------------------------
load_dotenv()
nvidia_api = os.getenv("NVIDIA_API_KEY")
assert nvidia_api, "⚠ ERROR: NVIDIA_API_KEY missing"

# ------------------------------------------------
# Streamlit UI Configuration
# ------------------------------------------------
st.set_page_config(page_title="AI Data Analyst", layout="wide")

st.markdown("""
<style>
.chat-card {
    background-color: #111827;
    padding: 18px;
    border-radius: 16px;
    margin-bottom: 14px;
    border: 1px solid #1f2937;
}
.sql-card {
    background-color: #0f172a;
    padding: 16px;
    border-radius: 14px;
    margin-top: 10px;
    border: 1px solid #1e293b;
}
.user-msg {color:#38bdf8; font-weight:600;}
.assistant-msg {color:#f9fafb; font-weight:500;}
.section-title {color:#22c55e; font-size:20px; font-weight:700;}
</style>
""", unsafe_allow_html=True)

st.title("💬 AI Data Analyst Chatbot")

# ------------------------------------------------
# Load CSV into SQLite (cached)
# ------------------------------------------------
@st.cache_data
def load_data():
    return pd.read_csv("Superstore.csv", encoding="latin1")

@st.cache_resource
def init_db(data):
    conn = sqlite3.connect("sales_temp.db", check_same_thread=False)
    data.to_sql("sales", conn, if_exists="replace", index=False)
    return conn

data = load_data()
conn = init_db(data)

# ------------------------------------------------
# Initialize NVIDIA LLM
# ------------------------------------------------
llm = ChatNVIDIA(
    model="meta/llama-3.1-70b-instruct",
    temperature=0.05,
    api_key=nvidia_api
)

# ------------------------------------------------
# LangGraph State
# ------------------------------------------------
class State(TypedDict):
    messages: List[Dict]
    intent: str
    sql_query: str
    sql_results: Any
    insight: str
    graph: Any

# ------------------------------------------------
# Node 1 — Intent Detection
# ------------------------------------------------
def intent_node(state: State):
    user_msg = state["messages"][-1]["content"]

    prompt = f"""
Identify user intent as:

SQL → If asking for numbers, totals, trends, grouped values, time-based analysis.
INSIGHT → If asking for explanation, reasoning, patterns, or interpretation.

Return ONLY: SQL or INSIGHT.

User query: "{user_msg}"
"""

    intent = llm.invoke([{"role": "user", "content": prompt}]).content.strip().upper()
    return {**state, "intent": intent}

# ------------------------------------------------
# Node 2 — SQL Generator
# ------------------------------------------------
def sql_node(state: State):
    if state["intent"] != "SQL":
        return state

    user_msg = state["messages"][-1]["content"]
    columns = ", ".join([f'"{c}"' for c in data.columns])

    prompt = f"""
You are an expert SQLite query generator.
Return ONLY a SQLite query. No explanation.

TABLE: sales
COLUMNS: {columns}

DATE FORMAT ("Order Date" dd/mm/yyyy → YYYY-MM-DD):
SUBSTR("Order Date",7,4)||'-'||SUBSTR("Order Date",4,2)||'-'||SUBSTR("Order Date",1,2)

User query:
{user_msg}
"""

    sql = llm.invoke([{"role": "user", "content": prompt}]).content.strip()

    # Clean before SELECT
    if "select" in sql.lower():
        sql = sql[sql.lower().index("select"):].strip()

    return {**state, "sql_query": sql}

# ------------------------------------------------
# Node 3 — SQL Executor
# ------------------------------------------------
def sql_exec_node(state: State):
    if not state.get("sql_query"):
        return state

    sql = state["sql_query"]

    try:
        df = pd.read_sql_query(sql, conn)

        # Auto graph building
        fig = None
        numeric = df.select_dtypes(include="number").columns

        if not df.empty and len(numeric) > 0:
            fig = px.line(
                df,
                y=numeric,
                x=df.index,
                markers=True,
                title="📈 Auto-Generated Data Visualization"
            )

        return {
            **state,
            "sql_results": df,
            "graph": fig
        }

    except Exception as e:
        return {**state, "sql_results": pd.DataFrame({"error": [str(e)]})}

# ------------------------------------------------
# Node 4 — Insights
# ------------------------------------------------
def insight_node(state: State):
    user_msg = state["messages"][-1]["content"]
    df = state.get("sql_results")

    prompt = f"""
Provide a clear business insight summary.

User question:
{user_msg}

SQL Results:
{df.head(30).to_string(index=False)}

Include:
- Trends
- Peaks & lows
- Possible reasons
- Business interpretation
"""

    insight = llm.invoke([{"role": "user", "content": prompt}]).content.strip()
    return {**state, "insight": insight}

# ------------------------------------------------
# Node 5 — Final
# ------------------------------------------------
def output_node(state: State):
    out = f"### 🧠 Insights\n{state['insight']}"
    return {**state, "messages": state["messages"] + [{"role": "assistant", "content": out}]}

# ------------------------------------------------
# Build LangGraph
# ------------------------------------------------
graph = StateGraph(State)
graph.add_node("intent", intent_node)
graph.add_node("sql_gen", sql_node)
graph.add_node("sql_exec", sql_exec_node)
graph.add_node("insight", insight_node)
graph.add_node("output", output_node)

graph.set_entry_point("intent")
graph.add_edge("intent", "sql_gen")
graph.add_edge("sql_gen", "sql_exec")
graph.add_edge("sql_exec", "insight")
graph.add_edge("insight", "output")
graph.add_edge("output", END)

app = graph.compile()

# ------------------------------------------------
# Session Memory
# ------------------------------------------------
if "conversation" not in st.session_state:
    st.session_state.conversation = []

# ------------------------------------------------
# Chat UI
# ------------------------------------------------
st.divider()
st.subheader("💭 Ask Something About Your Data")

col1, col2 = st.columns([8, 2])
with col1:
    user_input = st.text_input("Type your question...", key="user_input_box")
with col2:
    send = st.button("Send 🚀")

if send and user_input:
    st.session_state.conversation.append({"role": "user", "content": user_input})
    result = app.invoke({"messages": st.session_state.conversation})
    st.session_state.conversation.append(
        {"role": "assistant", "content": result["messages"][-1]["content"],
         "sql_results": result["sql_results"],
         "graph": result["graph"]}
    )

# ------------------------------------------------
# Display Chat Messages
# ------------------------------------------------
for msg in reversed(st.session_state.conversation):
    if msg["role"] == "user":
        st.markdown(f"<div class='chat-card user-msg'>🙋 {msg['content']}</div>", unsafe_allow_html=True)
    else:
        st.markdown(f"<div class='chat-card assistant-msg'>{msg['content']}</div>", unsafe_allow_html=True)

        # Show graph
        if msg.get("graph"):
            st.plotly_chart(msg["graph"], use_container_width=True)

        # SQL Table Button
        if "sql_results" in msg:
            if st.button("📄 Show SQL Results Table", key=str(id(msg))):
                st.markdown("<div class='section-title'>SQL Query Output</div>", unsafe_allow_html=True)
                st.markdown("<div class='sql-card'>", unsafe_allow_html=True)
                st.dataframe(msg["sql_results"], use_container_width=True)
                st.markdown("</div>", unsafe_allow_html=True)

    st.divider()
